## City Recommender app:

  In this notebook, i've tried to implement recommender engine which recommends a city based on features/attributes which you've shown interest. I've even deployed to the cloud so that anyone can access.

In [0]:
import pandas as pd
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity
np.random.seed(42)

In [2]:
#Scraping data /gathering data
df = pd.read_html('https://www.nestpick.com/millennial-city-ranking-2018/', skiprows=0)[0]
df.columns = ['#','city', 'country','Employment Score', 'Startup Score', 'Tourism Score','Housing Score', 'Food Ranking', 'Transport Score', 'Health Rank', 'Internet Speed Score', 'University Score', 'Access to Contraceptive Score', 'Gender Equality Score', 'Immigration Tolerence', 'Personal Freedom and Choice', 'LGBT friendly Score', 'Nightlife Score', 'Beer Ranking', 'Festival Ranking', 'Total']
# df.set_index('city',inplace=True)
df.drop('#',axis=1, inplace=True)
# df.index.name= None
df.head()

,city,country,Employment Score,Startup Score,Tourism Score,Housing Score,Food Ranking,Transport Score,Health Rank,Internet Speed Score,University Score,Access to Contraceptive Score,Gender Equality Score,Immigration Tolerence,Personal Freedom and Choice,LGBT friendly Score,Nightlife Score,Beer Ranking,Festival Ranking,Total
0,Berlin,Germany,4.8,9.0,6.2,5.2,7.2,7.1,3.9,5.0,4.9,6.9,7.5,8.4,7.8,9.7,10.0,7.0,8.0,7.14
1,Montréal,Canada,6.0,8.1,5.7,6.4,5.1,3.6,7.4,5.1,5.8,8.6,7.0,9.9,8.1,9.6,8.8,5.1,9.0,7.10
2,London,United Kingdom,6.6,9.8,9.8,0.5,2.2,7.2,4.4,4.7,10.0,9.5,7.1,8.7,9.6,9.4,9.4,2.5,7.5,6.71
3,Amsterdam,Netherlands,7.6,8.3,8.7,0.6,1.7,5.7,6.9,4.5,5.0,7.9,5.5,8.5,10.0,9.9,8.6,4.8,9.7,6.63
4,Toronto,Canada,6.3,9.5,4.4,2.7,6.1,4.3,7.4,7.1,5.2,8.6,7.0,9.9,9.4,9.8,4.5,3.6,9.4,6.60


In [0]:
df.to_csv('city_ranking.csv', index=False)

Since i'm gonna use only city feature ,so i'll remove country attribute from the dataset.Also i'm not sure i'll need total feature. Let's remove them:


In [4]:
#Remove unwanted attributes
data = df.set_index('city'). iloc[:,1:-1]
data.head()

,Employment Score,Startup Score,Tourism Score,Housing Score,Food Ranking,Transport Score,Health Rank,Internet Speed Score,University Score,Access to Contraceptive Score,Gender Equality Score,Immigration Tolerence,Personal Freedom and Choice,LGBT friendly Score,Nightlife Score,Beer Ranking,Festival Ranking
city,,,,,,,,,,,,,,,,,
Berlin,4.8,9.0,6.2,5.2,7.2,7.1,3.9,5.0,4.9,6.9,7.5,8.4,7.8,9.7,10.0,7.0,8.0
Montréal,6.0,8.1,5.7,6.4,5.1,3.6,7.4,5.1,5.8,8.6,7.0,9.9,8.1,9.6,8.8,5.1,9.0
London,6.6,9.8,9.8,0.5,2.2,7.2,4.4,4.7,10.0,9.5,7.1,8.7,9.6,9.4,9.4,2.5,7.5
Amsterdam,7.6,8.3,8.7,0.6,1.7,5.7,6.9,4.5,5.0,7.9,5.5,8.5,10.0,9.9,8.6,4.8,9.7
Toronto,6.3,9.5,4.4,2.7,6.1,4.3,7.4,7.1,5.2,8.6,7.0,9.9,9.4,9.8,4.5,3.6,9.4


In [5]:
#Convert it into integers
scores = df.set_index('city'). iloc[:,1:-1].round().astype(int)
scores.head()

,Employment Score,Startup Score,Tourism Score,Housing Score,Food Ranking,Transport Score,Health Rank,Internet Speed Score,University Score,Access to Contraceptive Score,Gender Equality Score,Immigration Tolerence,Personal Freedom and Choice,LGBT friendly Score,Nightlife Score,Beer Ranking,Festival Ranking
city,,,,,,,,,,,,,,,,,
Berlin,5,9,6,5,7,7,4,5,5,7,8,8,8,10,10,7,8
Montréal,6,8,6,6,5,4,7,5,6,9,7,10,8,10,9,5,9
London,7,10,10,0,2,7,4,5,10,10,7,9,10,9,9,2,8
Amsterdam,8,8,9,1,2,6,7,4,5,8,6,8,10,10,9,5,10
Toronto,6,10,4,3,6,4,7,7,5,9,7,10,9,10,4,4,9


### **Why Cosine_Similarity?**

   `Cosine similarity is a metric used to determine how similar the documents are irrespective of their size.`

   That's how similarity between two vectors being calculated 
   <img src='https://www.machinelearningplus.com/wp-content/uploads/2018/10/Cosine-Similarity-Formula-1-1024x133.png?ezimgfmt=ng:webp/ngcb1'>


  If you'd wanna learn more about it,visit <a href='https://www.machinelearningplus.com/nlp/cosine-similarity/'>Here!</a>

In [6]:
#Lets calculate similarities 
cosine_similarity(scores)

array([[1.        , 0.97713728, 0.92872247, ..., 0.76064317, 0.75167047,
        0.71010911],
       [0.97713728, 1.        , 0.93991451, ..., 0.81023932, 0.79920914,
        0.79407069],
       [0.92872247, 0.93991451, 1.        , ..., 0.71449984, 0.78578921,
        0.6591375 ],
       ...,
       [0.76064317, 0.81023932, 0.71449984, ..., 1.        , 0.86622045,
        0.78358256],
       [0.75167047, 0.79920914, 0.78578921, ..., 0.86622045, 1.        ,
        0.76641919],
       [0.71010911, 0.79407069, 0.6591375 , ..., 0.78358256, 0.76641919,
        1.        ]])

Hmm.. Above cell output can be intimidating/hard to interpret the results. But we'll prettify it.

In [7]:
#Prettifying
cos = pd.DataFrame(cosine_similarity(scores), columns=scores.index, index = scores.index)
cos

city,Berlin,Montréal,London,Amsterdam,Toronto,Vancouver,Barcelona,New York City,Cologne,Manchester,Hamburg,Bristol,San Francisco,Austin,Paris,Miami,Munich,Lisbon,Glasgow,Madrid,Melbourne,Ottawa,Birmingham,Sydney,Prague,Vienna,Québec City,Auckland,Los Angeles,São Paulo,Frankfurt,Buenos Aires,Mexico City,Portland,Chicago,Nantes,Budapest,Copenhagen,Santiago de Chile,Bogotá,...,Monterrey,Seattle,Brisbane,"Washington, D.C.",The Hague,Shanghai,Utrecht,Seville,Tel Aviv,Montpellier,Tokyo,Warsaw,Bordeaux,Singapore,Istanbul,Marseille,Wellington,Christchurch,Granada,Gothenburg,Lima,Cork,Groningen,Canberra,Brno,Rome,Osaka,Bucharest,Perth,Athens,Milan,Venice,Bergen,Cape Town,Innsbruck,Bern,Jerusalem,Brugge,Geneva,Bologna
city,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
Berlin,1.000000,0.977137,0.928722,0.950510,0.949095,0.918103,0.958486,0.920974,0.959179,0.949931,0.934774,0.879794,0.908838,0.902865,0.901236,0.912994,0.976779,0.949487,0.891972,0.941870,0.929800,0.885787,0.911135,0.926287,0.884834,0.975880,0.841736,0.926792,0.906976,0.875790,0.922823,0.865006,0.835738,0.905856,0.911135,0.888106,0.770370,0.918765,0.872266,0.792722,...,0.714392,0.855333,0.911825,0.850930,0.886729,0.757114,0.853023,0.818885,0.819169,0.863117,0.840501,0.782099,0.870296,0.822444,0.772072,0.853827,0.817378,0.777713,0.783312,0.854252,0.763518,0.806097,0.799731,0.785801,0.759425,0.815816,0.772692,0.757026,0.810720,0.767959,0.791236,0.719515,0.707497,0.765805,0.779153,0.731924,0.728269,0.760643,0.751670,0.710109
Montréal,0.977137,1.000000,0.939915,0.964724,0.971958,0.951885,0.962462,0.920323,0.941583,0.961343,0.942477,0.913961,0.913472,0.904747,0.920303,0.915982,0.956432,0.929912,0.921914,0.929572,0.956226,0.921672,0.927074,0.955281,0.833547,0.973368,0.882919,0.950146,0.920029,0.852739,0.930061,0.861127,0.799752,0.919763,0.914964,0.923682,0.728102,0.943195,0.861598,0.771853,...,0.697802,0.875082,0.951978,0.858288,0.909265,0.728034,0.914913,0.839722,0.834983,0.890745,0.828912,0.717010,0.901962,0.838774,0.734829,0.891481,0.846115,0.813637,0.809483,0.869581,0.725367,0.844497,0.858439,0.836570,0.718944,0.843737,0.776428,0.688080,0.868435,0.722111,0.804577,0.790815,0.767168,0.751238,0.802558,0.788305,0.788009,0.810239,0.799209,0.794071
London,0.928722,0.939915,1.000000,0.968212,0.925974,0.902572,0.895878,0.977148,0.841949,0.921051,0.845974,0.808220,0.962951,0.860140,0.961388,0.916381,0.930615,0.818103,0.839067,0.859105,0.946969,0.855369,0.841811,0.949864,0.751525,0.931485,0.770656,0.894170,0.944230,0.758473,0.865498,0.730156,0.685781,0.888925,0.909513,0.838111,0.629132,0.930247,0.765556,0.618111,...,0.572328,0.888162,0.896614,0.879499,0.849458,0.711185,0.885416,0.691887,0.791253,0.834480,0.803657,0.625170,0.828995,0.848807,0.635126,0.785784,0.764955,0.733328,0.628150,0.825551,0.552181,0.720609,0.734584,0.788148,0.550039,0.797862,0.686279,0.532946,0.856731,0.609014,0.812965,0.669057,0.709335,0.597920,0.660753,0.762223,0.684090,0.714500,0.785789,0.659137
Amsterdam,0.950510,0.964724,0.968212,1.000000,0.943879,0.927659,0.940543,0.955739,0.887645,0.931762,0.902976,0.848514,0.948293,0.899229,0.942469,0.949476,0.929535,0.858891,0.854321,0.884389,0.951823,0.888995,0.855617,0.953415,0.783768,0.968858,0.834033,0.901585,0.923626,0.753654,0.914292,0.758696,0.715912,0.920855,0.890485,0.883969,0.663333,0.927479,0.774760,0.640626,...,0.603866,0.902317,0.917257,0.875822,0.886027,0.685006,0.886071,0.747376,0.832226,0.855573,0.811325,0.667999,0.861275,0.828130,0.668317,0.823409,0.788193,0.744959,0.688077,0.801028,0.609695,0.753196,0.766843,0.782888,0.600107,0.849228,0.708634,0.572306,0.833384,0.651344,0.821885,0.748883,0.709102,0.627730,0.703653,0.784249,0.743160,0.739985,0.795076,0.735508
Toronto,0.949095,0.971958,0.925974,0.943879,1.000000,0.987980,0.928943,0.905178,0.890129,0.932596,0.909446,0.889869,0.924385,0.935408,0.880423,0.904148,0.952319,0.875118,0.937392,0.906125,0.932965,0.944955,0.904970,0.964835,0.771294,0.939332,0.897117,0.955742,0.92006

In [0]:
scores.index

Index(['Berlin', 'Montréal', 'London', 'Amsterdam', 'Toronto', 'Vancouver',
       'Barcelona', 'New York City', 'Cologne', 'Manchester',
       ...
       'Milan', 'Venice', 'Bergen', 'Cape Town', 'Innsbruck', 'Bern',
       'Jerusalem', 'Brugge', 'Geneva', 'Bologna'],
      dtype='object', name='city', length=110)

In [8]:
#Artificially create some interests which can later be used to generate recommendation:
user1 = np.random.choice(range(1,11),17).reshape(-1,17) #Reshape transforms it into two dimensional vector
user1

array([[ 7,  4,  8,  5,  7, 10,  3,  7,  8,  5,  4,  8,  8,  3,  6,  5,
         2]])

In [0]:
value = []
for index,city in enumerate(scores.index):
    city_old = scores.loc[city].values.reshape(-1,17)
    score = cosine_similarity(city_old, user1)
    value.append(score)

In [10]:
#similarity scores
value

[array([[0.89220417]]),
 array([[0.868623]]),
 array([[0.8759523]]),
 array([[0.85292868]]),
 array([[0.84175019]]),
 array([[0.81341556]]),
 array([[0.84212435]]),
 array([[0.86600226]]),
 array([[0.85371077]]),
 array([[0.90700116]]),
 array([[0.81325103]]),
 array([[0.76879045]]),
 array([[0.86254252]]),
 array([[0.8133641]]),
 array([[0.81350146]]),
 array([[0.82012368]]),
 array([[0.88076972]]),
 array([[0.87398682]]),
 array([[0.85679654]]),
 array([[0.85063264]]),
 array([[0.81284352]]),
 array([[0.85264026]]),
 array([[0.85620644]]),
 array([[0.83158741]]),
 array([[0.85837534]]),
 array([[0.88137286]]),
 array([[0.79562681]]),
 array([[0.86593748]]),
 array([[0.86272954]]),
 array([[0.80098475]]),
 array([[0.83657704]]),
 array([[0.77846585]]),
 array([[0.78030267]]),
 array([[0.8257734]]),
 array([[0.85620644]]),
 array([[0.81619864]]),
 array([[0.79903396]]),
 array([[0.79571214]]),
 array([[0.87723002]]),
 array([[0.71057267]]),
 array([[0.80414583]]),
 array([[0.78689532]]

In [11]:
similarity = pd.Series(value, index=scores.index)

#Below code outputs top recommendation for you
similarity.sort_values(ascending=False).astype(float).idxmax()

'Manchester'

Let's do create a function for above codes which recommends a city.

In [0]:
#Finding similarity between cities
def find_similarity(column, user, number):
    new_df = scores[column]
    value = []
    for index,city in enumerate(new_df.index):
        city_old = new_df.loc[city].values.reshape(-1,number)
        user = user.reshape(-1, number)
        score = cosine_similarity(city_old, user)
        value.append(score)
    similarity = pd.Series(value, index=scores.index)
    city_similar = similarity.sort_values(ascending=False).astype(float).idxmax()
    message = f'Based on your preference, {city_similar} is the best recommendation'
    return message

In [14]:
column = ['Tourism Score','Housing Score', 'Food Ranking'] #Choosing three particular columns

#NO of features
number = len(column)

#User defined interest
user = np.random.choice(range(1,10), number) # Choose 3 numbers between 1 to 10
print(find_similarity(column, user, number))

Based on your preference, Sydney is the best recommendation


In [16]:
word = 'Bologna'
if word in scores.index: #If given city present in df
    country = df.loc[df['city'] == word, 'country'].iloc[0] #country of that a city
    print(f'{word} is the capital of {country}')

    if word in df['city'].head().values:
        print('It is actually one of the top 5 cities that has pique millennial interests')

    elif word in df['city'].head(10).values:
        print('It is actually one of the top 10 cities that has pique millennial interests')

    elif word in df['city'].tail(5).values:
        print('It is actually one of the least 5 cities that has pique millennial interests')
    
else:
    print(f'Which country is {word} located in ?')

Bologna is the capital of Italy
It is actually one of the least 5 cities that has pique millennial interests


In [17]:
if word in scores.index:
    title = f'About {word}'
    subtitle = 'City Ranking in terms of Business, essentials, Openness and recreation score(over 10.0)'
    print(title)
#     print('\n')
    print(subtitle)
#     print('\n')
    for i , j in zip(list(data.loc['Berlin'].index),data.loc['Berlin']):
        print(f'{i} == {j}')

About Bologna
City Ranking in terms of Business, essentials, Openness and recreation score(over 10.0)
Employment Score == 4.8
Startup Score == 9.0
Tourism Score == 6.2
Housing Score == 5.2
Food Ranking == 7.2
Transport Score == 7.1
Health Rank == 3.9
Internet Speed Score == 5.0
University Score == 4.9
Access to Contraceptive Score == 6.9
Gender Equality Score == 7.5
Immigration Tolerence == 8.4
Personal Freedom and Choice == 7.8
LGBT friendly Score == 9.7
Nightlife Score == 10.0
Beer Ranking == 7.0
Festival Ranking == 8.0


In [18]:
sort_by_country = df[["city", "country"]].sort_values("country")
sort_by_country.head()

,city,country
31,Buenos Aires,Argentina
98,Perth,Australia
93,Canberra,Australia
23,Sydney,Australia
20,Melbourne,Australia


In [0]:
location = []
for index, city, country in sort_by_country.itertuples():
    new = f'{city}, {country}'
    location.append(new)

In [26]:
location

['Buenos Aires, Argentina',
 'Perth, Australia',
 'Canberra, Australia',
 'Sydney, Australia',
 'Melbourne, Australia',
 'Adelaide, Australia',
 'Brisbane, Australia',
 'Graz, Austria',
 'Innsbruck, Austria',
 'Vienna, Austria',
 'Brugge, Belgium',
 'Antwerp, Belgium',
 'Brussels, Belgium',
 'São Paulo, Brazil',
 'Rio de Janeiro, Brazil',
 'Montréal, Canada',
 'Toronto, Canada',
 'Ottawa, Canada',
 'Vancouver, Canada',
 'Québec City, Canada',
 'Santiago de Chile, Chile',
 'Shanghai, China',
 'Beijing, China',
 'Bogotá, Colombia',
 'Medellín, Colombia',
 'Brno, Czech Republic',
 'Prague, Czech Republic',
 'Copenhagen, Denmark',
 'Helsinki, Finland',
 'Marseille, France',
 'Toulouse, France',
 'Bordeaux, France',
 'Nantes, France',
 'Lyon, France',
 'Lille, France',
 'Montpellier, France',
 'Paris, France',
 'Berlin, Germany',
 'Hamburg, Germany',
 'Cologne, Germany',
 'Frankfurt, Germany',
 'Munich, Germany',
 'Athens, Greece',
 'Hong Kong, Hong Kong SAR',
 'Budapest, Hungary',
 'Cork, 

In [20]:
list(scores.index)

['Berlin',
 'Montréal',
 'London',
 'Amsterdam',
 'Toronto',
 'Vancouver',
 'Barcelona',
 'New York City',
 'Cologne',
 'Manchester',
 'Hamburg',
 'Bristol',
 'San Francisco',
 'Austin',
 'Paris',
 'Miami',
 'Munich',
 'Lisbon',
 'Glasgow',
 'Madrid',
 'Melbourne',
 'Ottawa',
 'Birmingham',
 'Sydney',
 'Prague',
 'Vienna',
 'Québec City',
 'Auckland',
 'Los Angeles',
 'São Paulo',
 'Frankfurt',
 'Buenos Aires',
 'Mexico City',
 'Portland',
 'Chicago',
 'Nantes',
 'Budapest',
 'Copenhagen',
 'Santiago de Chile',
 'Bogotá',
 'Valencia',
 'Bangkok',
 'Dublin',
 'Lyon',
 'Philadelphia',
 'Helsinki',
 'Antwerp',
 'Porto',
 'Beijing',
 'Stockholm',
 'Denver',
 'Toulouse',
 'Kuala Lumpur',
 'Medellín',
 'Rio de Janeiro',
 'Graz',
 'Hong Kong',
 'Edinburgh',
 'Guadalajara',
 'Lille',
 'Rotterdam',
 'Taipei',
 'Coventry',
 'Moscow',
 'Seoul',
 'Adelaide',
 'Brussels',
 'Zurich',
 'Boston',
 'Oslo',
 'Monterrey',
 'Seattle',
 'Brisbane',
 'Washington, D.C.',
 'The Hague',
 'Shanghai',
 'Utrecht'

In [21]:
if word in scores.index:
    title = f'About {word}'
    subtitle = 'City Ranking in terms of Business, essentials, Openness and recreation score(over 10.0)'
    print(title)
#     print('\n')
    country = df.loc[df['city'] == word, 'country'].iloc[0]
    print(f'{word} is the capital of {country}')
    if word in df['city'].head().values:
        print('It is actually one of the top 5 cities that has pique millennial interests')
    elif word in df['city'].head(10).values:
        print('It is actually one of the top 10 cities that has pique millennial interests')
    elif word in df['city'].tail(5).values:
        print('It is actually one of the least 5 cities that has pique millennial interests')
    print(subtitle)
#     print('\n')
    
    for i , j in zip(list(data.loc['Berlin'].index),data.loc['Berlin']):
        print(f'{i} == {j}')

About Bologna
Bologna is the capital of Italy
It is actually one of the least 5 cities that has pique millennial interests
City Ranking in terms of Business, essentials, Openness and recreation score(over 10.0)
Employment Score == 4.8
Startup Score == 9.0
Tourism Score == 6.2
Housing Score == 5.2
Food Ranking == 7.2
Transport Score == 7.1
Health Rank == 3.9
Internet Speed Score == 5.0
University Score == 4.9
Access to Contraceptive Score == 6.9
Gender Equality Score == 7.5
Immigration Tolerence == 8.4
Personal Freedom and Choice == 7.8
LGBT friendly Score == 9.7
Nightlife Score == 10.0
Beer Ranking == 7.0
Festival Ranking == 8.0


In [22]:
list(zip(list(data.loc[word].index),data.loc[word]))

[('Employment Score', 5.8),
 ('Startup Score', 1.4),
 ('Tourism Score', 3.2),
 ('Housing Score', 5.1),
 ('Food Ranking', 1.7),
 ('Transport Score', 1.1),
 ('Health Rank', 9.1),
 ('Internet Speed Score', 4.6),
 ('University Score', 1.0),
 ('Access to Contraceptive Score', 3.4),
 ('Gender Equality Score', 3.3),
 ('Immigration Tolerence', 6.2),
 ('Personal Freedom and Choice', 0.8),
 ('LGBT friendly Score', 2.0),
 ('Nightlife Score', 0.9),
 ('Beer Ranking', 2.3),
 ('Festival Ranking', 2.8)]

In [0]:
#Extracting cities best features to display how it might work for you
ranking = list(zip(list(data.loc[word].index),data.loc[word]))
breakdown = pd.DataFrame(ranking, columns = ['Category','Score'])

In [24]:
breakdown

,Category,Score
0,Employment Score,5.8
1,Startup Score,1.4
2,Tourism Score,3.2
3,Housing Score,5.1
4,Food Ranking,1.7
5,Transport Score,1.1
6,Health Rank,9.1
7,Internet Speed Score,4.6
8,University Score,1.0
9,Access to Contraceptive Score,3.4


In [25]:
word = location[1]
if word == 'others':
    great = scores[column]
else: 
    locat = word.split(',')
    great = scores[scores.index !=  locat[0]][column]
    print(locat)
great.head()

['Perth', ' Australia']


,Tourism Score,Housing Score,Food Ranking
city,,,
Berlin,6,5,7
Montréal,6,6,5
London,10,0,2
Amsterdam,9,1,2
Toronto,4,3,6


In [27]:
location[0]

'Buenos Aires, Argentina'

In [28]:
location[0].split(',')

['Buenos Aires', ' Argentina']

In [29]:
new_df = scores[column]
new_df.shape

(110, 3)

In [30]:
great.shape

(109, 3)